# Import the repository from GitHub

First of all we start by importing the repository that we stored in the github project.

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_1 = user_secrets.get_secret("NEW_GITHUB_TOKEN")

In [2]:
token = UserSecretsClient().get_secret("NEW_GITHUB_TOKEN")
! git clone https://{token}@github.com/madratak/DIQ_Project2024.git

Cloning into 'DIQ_Project2024'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 117 (delta 48), reused 27 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (117/117), 802.40 KiB | 8.36 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [3]:
%cd /kaggle/working/DIQ_Project2024

/kaggle/working/DIQ_Project2024


# Set up the dataset

At this point we can proceed by importing the correct libraries and then importing the data itself inside our notebook.

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import re

# Set max column width to None to display full content
pd.set_option('display.max_colwidth', None)

In [5]:
SERVICES = pd.read_csv('/kaggle/working/DIQ_Project2024/data/raw/Comune-di-Milano-Servizi-alla-persona-parrucchieri-estetisti.csv',sep=';',encoding='unicode_escape')
SERVICES.head()

,Tipo esercizio pa,Ubicazione,Tipo via,Via,Civico,Codice via,ZD,Prevalente,Superficie altri usi,Superficie lavorativa
0,NaN,LGO DEI GELSOMINI N. 10 (z.d. 6),LGO,DEI GELSOMINI,10,5394.0,6,NaN,NaN,55.0
1,NaN,PZA FIDIA N. 3 (z.d. 9),PZA,FIDIA,3,1144.0,9,CENTRO MASSAGGI RILASSANTI NON ESTETICI,2.0,28.0
2,NaN,VIA ADIGE N. 10 (z.d. 5),VIA,ADIGE,10,4216.0,5,CENTRO BENESSERE,2.0,27.0
3,NaN,VIA BARACCHINI FLAVIO N. 9 (z.d. 1),VIA,BARACCHINI FLAVIO,9,356.0,1,TRUCCO SEMIPERMANENTE,NaN,NaN
4,NaN,VIA BERGAMO N. 12 (z.d. 4),VIA,BERGAMO,12,3189.0,4,NaN,NaN,50.0


# Wrangling

In [6]:
SERVICES = SERVICES.rename(columns={
    'Tipo esercizio pa': 't_es',
    'Ubicazione': 'ubic',
    'Tipo via': 't_via',
    'Via': 'via',
    'Civico': 'civ',
    'Codice via': 'cod_via',
    'ZD': 'zd',
    'Prevalente': 'main',
    'Superficie altri usi': 'sup_alt',
    'Superficie lavorativa': 'sup_lav'
})

new_SERVICES = SERVICES.copy()

new_SERVICES.head()

,t_es,ubic,t_via,via,civ,cod_via,zd,main,sup_alt,sup_lav
0,NaN,LGO DEI GELSOMINI N. 10 (z.d. 6),LGO,DEI GELSOMINI,10,5394.0,6,NaN,NaN,55.0
1,NaN,PZA FIDIA N. 3 (z.d. 9),PZA,FIDIA,3,1144.0,9,CENTRO MASSAGGI RILASSANTI NON ESTETICI,2.0,28.0
2,NaN,VIA ADIGE N. 10 (z.d. 5),VIA,ADIGE,10,4216.0,5,CENTRO BENESSERE,2.0,27.0
3,NaN,VIA BARACCHINI FLAVIO N. 9 (z.d. 1),VIA,BARACCHINI FLAVIO,9,356.0,1,TRUCCO SEMIPERMANENTE,NaN,NaN
4,NaN,VIA BERGAMO N. 12 (z.d. 4),VIA,BERGAMO,12,3189.0,4,NaN,NaN,50.0


## ZD

In [7]:
new_SERVICES.zd.unique()

array(['6', '9', '5', '1', '4', '2', '8', '3', nan, 'ACCONCIATORE', '7'],
      dtype=object)

In [8]:
new_SERVICES.loc[32:33].head()

,t_es,ubic,t_via,via,civ,cod_via,zd,main,sup_alt,sup_lav
32,Acconciatore,CSO COMO N. 15 interno club f. conti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,(z.d. 9),CSO,COMO,15,1111,9.0,ACCONCIATORE,NaN,195.0,NaN


Dropped the rows and added a new one.

In [9]:
# New merged row
merged_row = {
    "t_es": "Acconciatore",
    "ubic": "CSO COMO N. 15 (z.d. 9)",
    "t_via": "CSO",
    "via": "CSO COMO",
    "civ": "15",
    "cod_via": 1111.0,
    "zd": "9",
    "main": "ACCONCIATORE",
    "sup_alt": np.nan,
    "sup_lav": 195.0,
}

# Remove row at index 33 and replace row at index 32
new_SERVICES = new_SERVICES.drop(index=33, errors='ignore')  # Drop row 33
new_SERVICES.loc[32] = merged_row  # Add the merged row at index 32

# Ensure correct row order and reset index if necessary
new_SERVICES = new_SERVICES.sort_index().reset_index(drop=True)

new_SERVICES.loc[32:33].head()

,t_es,ubic,t_via,via,civ,cod_via,zd,main,sup_alt,sup_lav
32,Acconciatore,CSO COMO N. 15 (z.d. 9),CSO,CSO COMO,15,1111.0,9,ACCONCIATORE,NaN,195.0
33,Acconciatore,CSO DI PORTA NUOVA N. 46 ; (z.d. 1),CSO,DI PORTA NUOVA,46,1044.0,1,NaN,NaN,34.0


In [10]:
new_SERVICES.zd = new_SERVICES.zd.astype(str)
np.sort(new_SERVICES['zd'].unique())

array(['1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype=object)

## T_ES

In [11]:
new_SERVICES.t_es.unique()

array([nan, 'Acconciatore', 'ACCONCIATORE',
       'ACCONCIATORE;TIPO A ESTETICA MANUALE',
       'ACCONCIATORE;TIPO A ESTETICA MANUALE;TIPO B CENTRO DI ABBRONZATURA',
       'Acconciatore;TIPO A - REG.2003', 'ACCONCIATORE;TIPO A - REG.2003',
       'Acconciatore;TIPO A - REG.2003;TIPO B CENTRO DI ABBRONZATURA',
       'ACCONCIATORE;TIPO A - REG.2003;TIPO B CENTRO DI ABBRONZATURA',
       'BARBIERE', 'Centro abbronzatura', 'Centro benessere',
       'Centro benessere;TIPO A - REG.2003', 'Centro massaggi',
       'Centro massaggi;Centro benessere',
       'esecuzione di tatuaggi e piercing',
       'Esecuzione di tatuaggi e piercing', 'Estetista',
       'Estetista in profumeria;TIPO A ESTETICA MANUALE;TIPO D ESTET.APPAR.ELETTROMECC;TIPO C TRATT.ESTETICI DIMAGRIM',
       'Estetista;Manicure;Pedicure estetico',
       'Estetista;Parrucchiere per signora',
       'Estetista;Pedicure estetico', 'Manicure;Parrucchiere per signora',
       'Parrucchiere misto', 'Parrucchiere misto;TIPO A ES

In [12]:
# Ensure 't_es' column does not have NaN values and is treated as strings
new_SERVICES["t_es"] = new_SERVICES["t_es"].fillna("").astype(str)

# Split the 't_es' column by ';' and flatten the resulting lists
flattened_labels = [label.strip() for item in new_SERVICES["t_es"] for label in item.split(";")]

# Get the unique labels and sort them
unique_labels = sorted(set(flattened_labels))

# Initialize a dictionary to store counts
label_counts = {label: 0 for label in unique_labels}

# Loop through each row in the "t_es" column and count the occurrences of each label
for entry in new_SERVICES['t_es'].dropna():  # Drop NaN values to avoid errors
    for label in unique_labels:
        if label in entry:
            label_counts[label] += 1

# Print the counts
for label, count in label_counts.items():
    if label != "":
        print(f"{label}: {count}")

ACCONCIATORE: 717
Acconciatore: 190
BARBIERE: 2
Centro abbronzatura: 2
Centro benessere: 29
Centro massaggi: 139
Esecuzione di tatuaggi e piercing: 5
Estetista: 31
Estetista in profumeria: 2
Manicure: 3
Parrucchiere misto: 105
Parrucchiere per signora: 1213
Parrucchiere per uomo: 518
Pedicure estetico: 22
TIPO A - REG.2003: 865
TIPO A ESTETICA MANUALE: 263
TIPO A-B-C-D: 88
TIPO B CENTRO DI ABBRONZATURA: 603
TIPO C TRATT.ESTETICI DIMAGRIM: 138
TIPO D ESTET.APPAR.ELETTROMECC: 74
Truccatore: 1
esecuzione di tatuaggi e piercing: 19


In [13]:
# Mapping of original values to new values
label_mapping = {
    "ACCONCIATORE": "Acconciatore",
    "Acconciatore": "Acconciatore",
    "BARBIERE": "Parrucchiere per Uomo",
    "Centro abbronzatura": "Tipo B - Centro di Abbronzatura",
    "Centro benessere": "Centro Benessere",
    "Centro massaggi": "Centro Massaggi",
    "Esecuzione di tatuaggi e piercing": "Esecuzione di Tatuaggi e Piercing",
    "esecuzione di tatuaggi e piercing": "Esecuzione di Tatuaggi e Piercing",
    "Estetista": "Tipo A - Estetica Manuale",
    "Estetista in profumeria": "Tipo A - Estetica Manuale",
    "Manicure": "Tipo A - Estetica Manuale", # Assuming merge because there are just 4 'Manicure' values
    "Pedicure estetico": "Tipo A - Estetica Manuale",
    "Parrucchiere per signora": "Parrucchiere per Donna",
    "Parrucchiere per uomo": "Parrucchiere per Uomo",
    "Truccatore": "Truccatore",
    "TIPO A ESTETICA MANUALE": "Tipo A - Estetica Manuale",
    "TIPO A - REG.2003": "Tipo A - Estetica Manuale",
    "TIPO B CENTRO DI ABBRONZATURA": "Tipo B - Centro di Abbronzatura",
    "TIPO C TRATT.ESTETICI DIMAGRIM": "Tipo C - Trattamenti Estetici Dimagrimento",
    "TIPO D ESTET.APPAR.ELETTROMECC": "Tipo D - Estetica Apparati Elettromeccanici",
}

# Define a function for handling composite labels within a cell
def handle_composite_labels(value):
    # If "TIPO A-B-C-D" exists, split and expand into individual components
    if "TIPO A-B-C-D" in value:
        value = value.replace("TIPO A-B-C-D", "; ".join([
            "Tipo A - Estetica Manuale", 
            "Tipo B - Centro di Abbronzatura",
            "Tipo C - Trattamenti Estetici Dimagrimento", 
            "Tipo D - Estetica Apparati Elettromeccanici"
        ]))
    if "Parrucchiere misto" in value:
        value = value.replace("Parrucchiere misto", "; ".join([
            "Parrucchiere per Uomo",
            "Parrucchiere per Donna"
        ]))
    
    return value

# Apply composite label handling first
new_SERVICES["t_es"] = new_SERVICES["t_es"].apply(handle_composite_labels)

# Define a function to replace labels using the mapping
def replace_multiple_labels(cell_value, label_mapping):
    # Split the cell value by a delimiter (semicolon)
    labels = cell_value.split(';')  # Adjust delimiter as necessary
    # Strip whitespace and replace each label using the mapping
    labels = [label_mapping.get(label.strip(), label.strip()) for label in labels]
    
    # Remove duplicates by converting to a set and then back to a list
    labels = list(set(labels))  # Set ensures uniqueness, list keeps order
    # Rejoin the labels with the same delimiter
    return '; '.join(labels)

# Now apply the mapping for individual labels
new_SERVICES["t_es"] = new_SERVICES["t_es"].apply(lambda x: replace_multiple_labels(x, label_mapping))

# Check the updated column
# prova["t_es"].unique()

In [14]:
# Ensure 't_es' column does not have NaN values and is treated as strings
new_SERVICES["t_es"] = new_SERVICES["t_es"].fillna("").astype(str)

# Split the 't_es' column by ';' and flatten the resulting lists
flattened_labels = [label.strip() for item in new_SERVICES["t_es"] for label in item.split(";")]

# Get the unique labels and sort them
unique_labels = sorted(set(flattened_labels))

# Print the unique labels
print("Unique Labels:")
for label in unique_labels:
    print(label)

Unique Labels:

Acconciatore
Centro Benessere
Centro Massaggi
Esecuzione di Tatuaggi e Piercing
Parrucchiere per Donna
Parrucchiere per Uomo
Tipo A - Estetica Manuale
Tipo B - Centro di Abbronzatura
Tipo C - Trattamenti Estetici Dimagrimento
Tipo D - Estetica Apparati Elettromeccanici
Truccatore


In [15]:
# Initialize a dictionary to store counts
label_counts = {label: 0 for label in unique_labels}

# Loop through each row in the "t_es" column and count the occurrences of each label
for entry in new_SERVICES['t_es'].dropna():  # Drop NaN values to avoid errors
    for label in unique_labels:
        if label in entry:
            label_counts[label] += 1

# Print the counts
for label, count in label_counts.items():
    if label != "":
        print(f"{label}: {count}")

Acconciatore: 907
Centro Benessere: 29
Centro Massaggi: 139
Esecuzione di Tatuaggi e Piercing: 24
Parrucchiere per Donna: 1294
Parrucchiere per Uomo: 601
Tipo A - Estetica Manuale: 1266
Tipo B - Centro di Abbronzatura: 693
Tipo C - Trattamenti Estetici Dimagrimento: 226
Tipo D - Estetica Apparati Elettromeccanici: 162
Truccatore: 1


Fix values of "main" column.

In [16]:
new_SERVICES["main"].unique()

array([nan, 'CENTRO MASSAGGI RILASSANTI NON ESTETICI', 'CENTRO BENESSERE',
       'TRUCCO SEMIPERMANENTE', 'ESTETICA TIPO A', 'CENTRO MASSAGGI',
       'TATUAGGI E PIERCING', 'SERVIZI DI CENTRI PER IL BENESSERE FISICO',
       'ESTETICA A', 'acconciatore', 'ACCONCIATORE',
       'SERVIZI DEI SALONI DI BARBIERE E PARRUCCHIERE', 'Acconciatore',
       'PARRUCCHIERE UOMO DONNA BARBIERE', 'PARRUCCHIERE', 'ACONCIATORE',
       'ACCONCIATORE UOMO DONNA', 'acconciatore.',
       'ACCONCIATORE E ESTETICA',
       'CENTRO BENESSERE RILASSANTE NON ESTETICO',
       'CENTRO PER IL BENESSERE FISICO',
       'CENTTRO BENESSERE (ESCLUSI GLI STABILIMENTI TERMALI)',
       'SERVIZI DEI CENTRI E STABIL. PER IL BENESSERE FISICO',
       'STUDIO OLISTICO', 'CENTRO OLISTICO', 'centro massaggi',
       'MASSAGGI RILASSANTI', 'CENTRO MASSAGI RILASSANTI NON ESTETICI',
       'CENTRO MASSAGGI TUINA', 'CENTRO MASSAGGI RILASSANTI',
       'SERVIZI DI CENTRI PER IL BENESSERE',
       "ATTIVITA' DI MASSAGGI RILAS

In [17]:
# Ensure 't_es' column does not have NaN values and is treated as strings
new_SERVICES["main"] = new_SERVICES["main"].fillna("").astype(str)

mapping = {
    'CENTRO MASSAGGI RILASSANTI NON ESTETICI': 'Centro Massaggi',
    'CENTRO MASSAGGI': 'Centro Massaggi',
    'MASSAGGI RILASSANTI': 'Centro Massaggi',
    'CENTRO MASSAGGI TUINA': 'Centro Massaggi',
    'CENTRO MASSAGGI RILASSANTI': 'Centro Massaggi',
    'CENTRO MASSAGI RILASSANTI NON ESTETICI': 'Centro Massaggi', 
    "ATTIVITA' DI MASSAGGI RILASSANTI": 'Centro Massaggi',
    'MASSAGGI': 'Centro Massaggi',
    'centro massaggi': 'Centro Massaggi',
    'CENTRO MASSAGGI THAI RILASSANTI NON CURATIVI': 'Centro Massaggi',
    'CENTRO MASSAGGI RILASSANTI ESTETICI': 'Tipo A - Estetica Manuale; Centro Massaggi',
    'CENTRO BENESSERE': 'Centro Benessere',
    'centro benessere': 'Centro Benessere',
    'CENTRO PER IL BENESSERE FISICO': 'Centro Benessere',
    'CENTTRO BENESSERE (ESCLUSI GLI STABILIMENTI TERMALI)': 'Centro Benessere',
    'CENTRO BENESSERE RILASSANTE NON ESTETICO': 'Centro Benessere',
    'SERVIZI DI CENTRI PER IL BENESSERE FISICO': 'Centro Benessere',
    'SERVIZI DI CENTRI PER IL BENESSERE': 'Centro Benessere',
    'SERVIZI DEI CENTRI E STABIL. PER IL BENESSERE FISICO': 'Centro Benessere',
    'STUDIO OLISTICO': 'Centro Massaggi',
    'CENTRO OLISTICO': 'Centro Massaggi',
    'SERVIZI ORGANIZZATI CON SERVIZI DI SEGRETERIA': 'Tipo A - Estetica Manuale; Centro Massaggi',
    'TRUCCO SEMIPERMANENTE': 'Truccatore',
    'TRUCCATORE': 'Truccatore',
    'TATUAGGI E PIERCING': 'Esecuzione di Tatuaggi e Piercing',
    'ESECUZIONE DI TATUAGGI E PIERCING': 'Esecuzione di Tatuaggi e Piercing',
    'TATUAGGIO': 'Esecuzione di Tatuaggi e Piercing',
    'TATUAGGIO E PIERCING': 'Esecuzione di Tatuaggi e Piercing',
    'ESECUZIONE DI TATUAGGI': 'Esecuzione di Tatuaggi e Piercing',
    'ESECUZIONE TATUAGGI': 'Esecuzione di Tatuaggi e Piercing',
    'tatuaggi e piercing': 'Esecuzione di Tatuaggi e Piercing',
    'ACCONCIATORE': 'Acconciatore',
    'acconciatore': 'Acconciatore',
    'acconciatore.': 'Acconciatore',
    'ACONCIATORE': 'Acconciatore',
    'ACCONCIATORE UOMO DONNA': 'Acconciatore',
    'ACCONCIATORE E ESTETICA': 'Acconciatore; Tipo A - Estetica Manuale',
    'SERVIZI DEI SALONI DI BARBIERE E PARRUCCHIERE': 'Acconciatore',
    'PARRUCCHIERE UOMO DONNA BARBIERE': 'Acconciatore',
    'PARRUCCHIERE': 'Acconciatore',
    'ESTETISTA': 'Tipo A - Estetica Manuale',
    'ESTETICA': 'Tipo A - Estetica Manuale',
    'ESTETICA TIPO A': 'Tipo A - Estetica Manuale',
    'estetica tipo A': 'Tipo A - Estetica Manuale',
    'ESTETISTA MANICURE PEDICURE': 'Tipo A - Estetica Manuale',
    'estetista': 'Tipo A - Estetica Manuale',
    'estetica': 'Tipo A - Estetica Manuale',
    'APPLICAZIONE E DECORAZIONE UNGHIE ARTIFICIALE': 'Tipo A - Estetica Manuale',
    'RICOSTRUZIONE UNGHIE': 'Tipo A - Estetica Manuale',
    'ESTETICA - CENTRO BENESSERE': 'Tipo A - Estetica Manuale; Centro Benessere',
    'ESTETICA A': 'Tipo A - Estetica Manuale',
    'estetica A': 'Tipo A - Estetica Manuale',
    'ESTETICA a': 'Tipo A - Estetica Manuale',
    'ESTETICA A E SOLARIUM': 'Tipo A - Estetica Manuale; Tipo B - Centro di Abbronzatura',
    'solarium': 'Tipo B - Centro di Abbronzatura', 
    'ESTETICA A CON TATUAGGI': 'Tipo A - Estetica Manuale; Esecuzione di Tatuaggi e Piercing',
    'ESTETICA A e B': 'Tipo A - Estetica Manuale; Tipo B - Centro di Abbronzatura',
    'ESTETICA A B': 'Tipo A - Estetica Manuale; Tipo B - Centro di Abbronzatura',
    'ESTETICA A E B': 'Tipo A - Estetica Manuale; Tipo B - Centro di Abbronzatura',
    'estetica A e solarium': 'Tipo A - Estetica Manuale; Tipo B - Centro di Abbronzatura',
    'SOLARIUM': 'Tipo B - Centro di Abbronzatura',
    'SOLARIUM E ESTETICA DI TIPO A': 'Tipo A - Estetica Manuale; Tipo B - Centro di Abbronzatura',
    'ESTETICA TIPO A - B': 'Tipo A - Estetica Manuale; Tipo B - Centro di Abbronzatura',
    'CENTRO ESTETICO - SOLARIUN': 'Tipo A - Estetica Manuale; Tipo B - Centro di Abbronzatura'
}

def replace_multiple_labels(cell_value, label_mapping):
    # Split the cell value by a delimiter (semicolon)
    labels = cell_value.split(';')  # Adjust delimiter as necessary
    # Strip whitespace and replace each label using the mapping
    labels = [label_mapping.get(label.strip(), label.strip()) for label in labels]
    # Join the labels back with the same delimiter
    return ';'.join(labels)

# Now apply the mapping for individual labels
new_SERVICES["main"] = new_SERVICES["main"].apply(lambda x: replace_multiple_labels(x, mapping))

In [18]:
new_SERVICES["main"].unique()

array(['', 'Centro Massaggi', 'Centro Benessere', 'Truccatore',
       'Tipo A - Estetica Manuale', 'Esecuzione di Tatuaggi e Piercing',
       'Acconciatore', 'Acconciatore; Tipo A - Estetica Manuale',
       'Tipo A - Estetica Manuale; Centro Benessere',
       'Tipo A - Estetica Manuale; Centro Massaggi',
       'Tipo A - Estetica Manuale; Esecuzione di Tatuaggi e Piercing',
       'Tipo A - Estetica Manuale; Tipo B - Centro di Abbronzatura',
       'Tipo B - Centro di Abbronzatura'], dtype=object)

In [19]:
# Initialize a dictionary to store counts
flattened_labels = [label.strip() for item in new_SERVICES["main"] for label in item.split(";")]
# Get the unique labels and sort them
unique_main_labels = sorted(set(flattened_labels))

label_counts = {label: 0 for label in unique_main_labels}

# Loop through each row in the "t_es" column and count the occurrences of each label
for entry in new_SERVICES['main'].dropna():  # Drop NaN values to avoid errors
    for label in unique_main_labels:
        if label in entry:
            label_counts[label] += 1

# Print the counts
for label, count in label_counts.items():
    if label != "":
        print(f"{label}: {count}")

Acconciatore: 100
Centro Benessere: 13
Centro Massaggi: 98
Esecuzione di Tatuaggi e Piercing: 19
Tipo A - Estetica Manuale: 66
Tipo B - Centro di Abbronzatura: 20
Truccatore: 1


## MAIN
Move "main" values to "t_es" column in order to reduce the number of null values in "t_es" column. **(To add that we don't consider it very informative and it has too many values)**

In [20]:
# Ensure both columns are strings and handle NaN values
new_SERVICES['main'] = new_SERVICES['main'].fillna("").astype(str)
new_SERVICES['t_es'] = new_SERVICES['t_es'].fillna("").astype(str)

def update_t_es(row):
    main_values = row['main'].split(";") if row['main'] else []
    t_es_values = row['t_es'].split(";") if row['t_es'] else []
    
    # Remove extra whitespace around the labels
    main_values = [value.strip() for value in main_values]
    t_es_values = [value.strip() for value in t_es_values]
    
    # If t_es is empty, copy main values
    if not t_es_values:
        return ";".join(main_values)
    
    # Add main labels that are not already in t_es
    for main_value in main_values:
        # Add the condition: If main is "Acconciatore" and t_es contains "Parrucchiere per Uomo" or "Parrucchiere per Donna", don't add
        if main_value == "Acconciatore" and any(x in t_es_values for x in ["Parrucchiere per Uomo", "Parrucchiere per Donna"]):
            continue
        if main_value not in t_es_values:
            t_es_values.append(main_value)
    
    # Return the updated t_es column as a semicolon-separated string
    return ";".join(t_es_values)

# Apply the function row-wise
new_SERVICES['t_es'] = new_SERVICES.apply(update_t_es, axis=1)

In [21]:
new_SERVICES['t_es'] = new_SERVICES['t_es'].replace("", float("nan"))
print("The number of Nan in 't_es' column is {}.".format(sum(new_SERVICES["t_es"].isnull())))

The number of Nan in 't_es' column is 16.


Since "Main" info is incorporated in "t_es" column, we drop it.

In [22]:
new_SERVICES = new_SERVICES.drop('main', axis=1)

In [23]:
new_SERVICES.head()

,t_es,ubic,t_via,via,civ,cod_via,zd,sup_alt,sup_lav
0,NaN,LGO DEI GELSOMINI N. 10 (z.d. 6),LGO,DEI GELSOMINI,10,5394.0,6,NaN,55.0
1,Centro Massaggi,PZA FIDIA N. 3 (z.d. 9),PZA,FIDIA,3,1144.0,9,2.0,28.0
2,Centro Benessere,VIA ADIGE N. 10 (z.d. 5),VIA,ADIGE,10,4216.0,5,2.0,27.0
3,Truccatore,VIA BARACCHINI FLAVIO N. 9 (z.d. 1),VIA,BARACCHINI FLAVIO,9,356.0,1,NaN,NaN
4,NaN,VIA BERGAMO N. 12 (z.d. 4),VIA,BERGAMO,12,3189.0,4,NaN,50.0


Drop rows labeled as "Truccatore" in "t_es", since the class is under-represented.

In [24]:
new_SERVICES = new_SERVICES[new_SERVICES["t_es"] != "Truccatore"]

## T_VIA

In [25]:
SERVICES.t_via.unique()

array(['LGO', 'PZA', 'VIA', 'VLE', 'CSO', nan, 'COMO', 'PAS', 'PLE',
       'ALZ', 'BST', 'FOR', 'VIE', 'VLO', 'GLL', 'RIP', 'PTA', 'SIT'],
      dtype=object)

In [26]:
new_SERVICES.t_via.unique()

array(['LGO', 'PZA', 'VIA', 'VLE', 'CSO', 'PAS', 'PLE', 'ALZ', 'BST',
       'FOR', 'VIE', 'VLO', 'GLL', 'RIP', 'PTA', 'SIT'], dtype=object)

## COD_VIA

In [27]:
# Check if any value in 'cod_via' is not a float
non_float_values = SERVICES['cod_via'].apply(lambda x: not isinstance(x, float))
if non_float_values.any():
    print("There are non-float values in 'cod_via'.")
else:
    print("All values in 'cod_via' are floats.")

new_SERVICES.cod_via = new_SERVICES.cod_via.astype(str) 

All values in 'cod_via' are floats.


## CIV

In [28]:
new_SERVICES.civ.unique()

array(['10', '3', '12', '13', '39', '25', '6', '20', '30', '2', '67',
       '16', '111', '36', '83', '5', '22', '23', '1', '4', '91', '8',
       '19', '38', '15', '46', '54', '65', '103', '40', '29', '9', '32',
       '7', '17', '27', '12A', '48', '61', '47', '232', '234', '57', '75',
       '34', '150', '71', '55', '21', '6A', '163', '90', '14', '53', '70',
       '119F', '26', '24', '40945', '128', '3B', '41', '64', '68', '35',
       '49', '42', '99', '52', '72', '117', '79', '96', '58', '81', '112',
       '93/1', '87', '63', '18', '123', '31', nan, '33', '11', '242',
       '46D', '28', '80', '98', '59', '37', '43', '60', '50', '44', '246',
       '326', '407', '410', '51', '110', '130', '131', '157', '174', '62',
       '107', '73', '139', '147', '168', '213', '255', '279', '45', '77',
       '4A', '187', '89', '207', '227', '244', '56', '100', '104', '74',
       '66', '92', '108', '121', '76', '233', '249', '256', '267', '270',
       '82', '116', '236', '69', '178', '143', '

In [29]:
new_SERVICES[new_SERVICES["via"] == "CSO COMO"].head()

,t_es,ubic,t_via,via,civ,cod_via,zd,sup_alt,sup_lav
32,Acconciatore,CSO COMO N. 15 (z.d. 9),CSO,CSO COMO,15,1111.0,9,NaN,195.0


## VIA: Removing Redundant Information from Records

When processing records, some entries contain redundant information in the `via` column that matches part of the `t_via` column.
To ensure consistency and follow a specific order, we remove the redundant portion.

**Example**

Given the following data:

| t_es          | ubic                      | t_via | via      | civ | cod_via | zd | sup_alt | sup_lav |
|---------------|---------------------------|-------|----------|-----|---------|----|---------|---------|
| Acconciatore  | CSO COMO N. 15 (z.d. 9)  | CSO   | CSO COMO     | 15  | 1111.0  | 9  | NaN     | 195.0   |


The updated record becomes:

| t_es          | ubic                      | t_via | via      | civ | cod_via | zd | sup_alt | sup_lav |
|---------------|---------------------------|-------|----------|-----|---------|----|---------|---------|
| Acconciatore  | CSO COMO N. 15 (z.d. 9)  | CSO   | COMO     | 15  | 1111.0  | 9  | NaN     | 195.0   |


In this transformation:
- The `via` column value `CSO COMO` had the redundant prefix `CSO` matching the value in the `t_via` column.
- The prefix was removed, leaving only the unique portion: `COMO`.

In [30]:
new_SERVICES.via = new_SERVICES.via.fillna("").astype(str)
new_SERVICES.t_via = new_SERVICES.t_via.fillna("").astype(str) 

# Function to remove the redundant prefix
def clean_via(row):
    if row["via"].startswith(row["t_via"] + " ") and row["t_via"] != "":  # Check if via starts with t_via
        return row["via"].replace(row["t_via"] + " ", "")  # Remove t_via + space
    return row["via"]  # Return original if no match

# Apply the function to the 'via' column
new_SERVICES["via"] = new_SERVICES.apply(clean_via, axis=1)

new_SERVICES[new_SERVICES.via == "COMO"].head(1)

,t_es,ubic,t_via,via,civ,cod_via,zd,sup_alt,sup_lav
32,Acconciatore,CSO COMO N. 15 (z.d. 9),CSO,COMO,15,1111.0,9,NaN,195.0


## UBIC: Extraction and Structuring of Location Data
We process the `ubic` field to extract and organize its contents into distinct components: `ubic_t_via` (street type), `ubic_via` (street name), `ubic_civ` (street number), `ubic_zd` (zone description), and `note` (additional information). This step allows us to compare these extracted values with the corresponding columns that share similar semantic meanings (`t_via`, `via`, `civ`, and `zd`). 

Once the comparison is complete, we remove the `ubic` field and its derived columns to eliminate redundancy. 

This approach ensures a cleaner and more structured dataset for further analysis.

**Example**

Given the following data:

| t_es  | ubic | t_via | via | civ | cod_via | zd | sup_alt | sup_lav | 
|-------|------|-------|-----|-----|---------|----|---------|---------|
| Tipo A - Estetica Manuale;Tipo C - Trattamenti Estetici Dimagrimento | ALZ NAVIGLIO PAVESE N. 8 int.attivita' di parrucchiere; (z.d. 6) | ALZ | NAVIGLIO PAVESE | 8 | 5161.0 | 6 | NaN | NaN |

The updated record becomes:

| t_es  | ubic | t_via | via | civ | cod_via | zd | sup_alt | sup_lav | ubic_t_via | ubic_via | ubic_civ | ubic_zd | note |
|-------|------|-------|-----|-----|---------|----|---------|---------|------------|----------|----------|--------|------|
| Tipo A - Estetica Manuale;Tipo C - Trattamenti Estetici Dimagrimento | ALZ NAVIGLIO PAVESE N. 8 int.attivita' di parrucchiere; (z.d. 6) | ALZ | NAVIGLIO PAVESE | 8 | 5161.0 | 6 | NaN | NaN | ALZ | NAVIGLIO PAVESE | 8 | 6 | int.attivita' di parrucchiere |

**Explanation**:

In this transformation:
- The `via` column value `NAVIGLIO PAVESE` had the redundant prefix `ALZ` matching the value in the `t_via` column.
- The prefix `ALZ` is removed from the `via` column, leaving the unique portion: `NAVIGLIO PAVESE`.
- The `ubic` field remains the same in structure but is used to extract the relevant information, including the street name, number, and zone description, which are now placed in separate columns (`ubic_via`, `ubic_civ`, `ubic_zd`), while any additional information (e.g., `int.attivita' di parrucchiere`) is placed in the `note` column.

In [31]:
new_SERVICES['ubic'] = new_SERVICES['ubic'].fillna("").astype(str)

def split_ubicazione(ubicazione):
    # Initialize variables
    tipo_via = np.nan
    via = np.nan
    civico = np.nan
    zd = np.nan
    note = np.nan

    # Split the string into parts
    parts = ubicazione.split(" ")

    # Extract "Tipo Via" (first word if valid)
    if parts[0] in new_SERVICES.t_via.unique():
        tipo_via = parts[0]
        parts = parts[1:]  # Remove 'Tipo Via' from further processing

    # Find "Civico" (number after "N." or "num.")
    match = re.search(r'\b(?:N\.|num\.)\s*([\w]+)', ubicazione, re.IGNORECASE)
    if match:
        civico = match.group(1).strip(";")
        parts = re.sub(r'\b(?:N\.|num\.)\s*[\w]+', '', ubicazione).split()

    # Extract "ZD" (zone description)
    match_zd = re.search(r'\(z\.d\.\s*(\d+)\)', ubicazione, re.IGNORECASE)
    if match_zd:
        zd = match_zd.group(1)
        parts = re.sub(r'\(z\.d\.\s*\d+\)', '', " ".join(parts)).split()

    # Identify and extract the "Via" (remaining uppercase parts)
    via_parts = []
    note_parts = []
    for part in parts:
        if part.isupper():  # Uppercase letters are part of "Via"
            via_parts.append(part)
        else:  # Lowercase letters are considered "Note"
            note_parts.append(part)

    # Construct the "Via" without the "Tipo Via"
    via_parts = [part for part in via_parts if part != tipo_via]  # Remove 'tipo_via' from 'via_parts'
    via = " ".join(via_parts).strip()
    note = " ".join(note_parts).strip(";") if note_parts else np.nan
    if note == "":
        note = np.nan
    
    # Remove leading zeros unless the value contains alphabetic characters
    if civico and civico.isdigit():
        civico = str(int(civico))  # Remove leading zeros by converting to an integer
    elif civico:  # Preserve alphanumeric strings like '012B'
        civico = civico.lstrip("0")  # Strip leading zeros but keep characters intact

    return tipo_via, via, civico, zd, note

# Apply the function to create new columns
new_SERVICES[['ubic_t_via', 'ubic_via', 'ubic_civ', 'ubic_zd', 'note']] = new_SERVICES['ubic'].apply(lambda x: pd.Series(split_ubicazione(x)))

***To remove (we will do it directly at the end)***
Starting to remove some values. Just to see in the csv how the rows with valid note appear.

In [32]:
# Function to check conditions and set the values to empty if conditions are met
def update_columns(row):
    # Check if note is empty and the other columns match
    if pd.isna(row['note']) and row['civ'] == row['ubic_civ'] and row['via'] == row['ubic_via'] and row['zd'] == row['ubic_zd']:
        row['ubic_t_via'] = np.nan
        row['ubic_via'] = np.nan
        row['ubic_civ'] = np.nan
        row['ubic_zd'] = np.nan
    return row

# Apply the function to update the columns
new_SERVICES = new_SERVICES.apply(update_columns, axis=1)

# Check the result
new_SERVICES.head()

,t_es,ubic,t_via,via,civ,cod_via,zd,sup_alt,sup_lav,ubic_t_via,ubic_via,ubic_civ,ubic_zd,note
0,NaN,LGO DEI GELSOMINI N. 10 (z.d. 6),LGO,DEI GELSOMINI,10,5394.0,6,NaN,55.0,NaN,NaN,NaN,NaN,NaN
1,Centro Massaggi,PZA FIDIA N. 3 (z.d. 9),PZA,FIDIA,3,1144.0,9,2.0,28.0,NaN,NaN,NaN,NaN,NaN
2,Centro Benessere,VIA ADIGE N. 10 (z.d. 5),VIA,ADIGE,10,4216.0,5,2.0,27.0,NaN,NaN,NaN,NaN,NaN
4,NaN,VIA BERGAMO N. 12 (z.d. 4),VIA,BERGAMO,12,3189.0,4,NaN,50.0,NaN,NaN,NaN,NaN,NaN
5,Centro Benessere,VIA BOTTEGO VITTORIO N. 13 (z.d. 2),VIA,BOTTEGO VITTORIO,13,2521.0,2,5.0,90.0,NaN,NaN,NaN,NaN,NaN


In [33]:
#Qui dentro si potranno visualizzare meglio le colonne.
new_SERVICES.to_csv("new_final.csv")

In [34]:
SERVICES.civ.unique()

array(['10', '3', '9', '12', '13', '39', '25', '6', '20', '30', '2', '67',
       '16', '111', '36', '83', '5', '22', '23', '1', '4', '91', '8',
       '19', '38', nan, '1111', '46', '54', '65', '103', '40', '29', '32',
       '7', '17', '27', '12A', '48', '61', '47', '15', '232', '234', '57',
       '75', '34', '150', '71', '55', '21', '6A', '163', '90', '14', '53',
       '70', '119F', '26', '24', '40945', '128', '3B', '41', '64', '68',
       '35', '49', '42', '99', '52', '72', '117', '79', '96', '58', '81',
       '112', '93/1', '87', '63', '18', '123', '31', '33', '11', '242',
       '46D', '28', '80', '98', '59', '37', '43', '60', '50', '44', '246',
       '326', '407', '410', '51', '110', '130', '131', '157', '174', '62',
       '107', '73', '139', '147', '168', '213', '255', '279', '45', '77',
       '4A', '187', '89', '207', '227', '244', '56', '100', '104', '74',
       '66', '92', '108', '121', '76', '233', '249', '256', '267', '270',
       '82', '116', '236', '69', '178', 